In [1]:
%env CUDA_VISIBLE_DEVICES=3,5
%env TMPDIR=/raid/users/ryan_cheng/tmp
import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

np.random.seed(0)

from utils import *
import utils
from consistency_eval import *
from education_generation import *

try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass

env: CUDA_VISIBLE_DEVICES=3,5
env: TMPDIR=/raid/users/ryan_cheng/tmp
INFO 04-27 07:12:09 __init__.py:190] Automatically detected platform cuda.


In [2]:
with open(os.path.abspath('../ryan_openai.txt'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [3]:
filename = '/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/exp/04.22.25/Llama-3.1-8B-Instruct_0_500.json'

with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/config_therapy.json", 'r') as f:
    config_therapy = json.load(f)

with open(filename, 'r') as f:
    data = json.load(f)

with open("./config/education/gpt-4o-mini.json", 'r') as f:
    config_gpt4_mini = json.load(f)

for key, value in config_gpt4_mini.items():
    config[key] = value

for key, value in config_therapy.items():
    prompts[key] = value

In [4]:
with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/config/eval_prompts.json" , 'r') as f:
    eval_prompts = json.load(f)

In [123]:
config['eval_model'] = 'Llama-3.1-8B-Instruct'
config['gpus'] = 1

In [5]:
config['eval_model'] = 'Llama-3.1-70B-Instruct'
config['gpus'] = 2

In [125]:
config['eval_model'] = 'gpt-4o-mini'

In [6]:
config['task_name'] = 'Therapy'

In [7]:
eval_prompts

{'strategy_consistency': "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, provide 1 sentence explaining your reasoning based on the strategy, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s strategy is described as follows:\n%SPEAKER_STRATEGY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reasoning followed by YES or NO.\n\n",
 'background_consistency': "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. Provide 1 sentence explaining your reasoning based on the background, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consist

In [8]:
def extract_list(text):
    pattern = r'\[.*?\]'
    match = re.search(pattern, text)
    if match:
        try:
            return eval(match.group())
        except (SyntaxError, NameError):
            return []
    return[]

In [9]:
prompts['eval_prompts'] = eval_prompts

In [10]:
test_list = ["a", "b", "c", "d", "e", "f", "g"]

In [12]:
prompts["eval_prompts"]["index_consistency_background"]

"%SCENARIO_DESC% For the following line spoken by %SPEAKER_ROLE%, first determine if there is a CLEAR conflict or inconsistency between the line and any line within the conversation history spoken by %SPEAKER_ROLE%. IF there is a conflict, provide a sentence of reasoning followed by a list of indices of lines in the conversation history that have a clear conflict with the current line. Otherwise, provide a sentence of reasoning followed by an empty list. ONLY INCLUDE INDICES OF LINES THAT CORRESPOND TO %SPEAKER_ROLE%. The conversation up to this point is as follows: %CONVERSATION%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your reasoning as 1 sentence, followed by a list of indices of conflicting lines from the conversation history formatted like a Python list in the following format: [index1, index2, index3, ...].\n\n"

In [13]:
def format_conversation(conversation):
    return "".join([str(i) + ": " + line for i, line in conversation])

In [14]:
print(format_conversation(data[0]["conversation"][:3]))

0: Patient: I'm just here for some advice on how to manage my workload, I guess. Nothing too deep, just some practical tips to get me through the next few months.
1: Therapist: I appreciate your desire to start with a more practical approach, but I'm curious, is there something specific that's driving your desire to manage your workload right now, or is there another layer to your concerns that you're not sharing?
2: Patient: I don't think there's anything specific driving my desire to manage my workload, I just feel like I'm falling behind and I want to get everything under control. Can we just focus on the strategies I can use to prioritize tasks and manage my time better?



In [15]:
def eval_index_consistency(conv_dict, both_agents=False):
    conv_dict['eval_index_consistency'] = []
    conv_dict['P1_index_consistency_score'] = 0
    if both_agents:
        conv_dict['P2_index_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0
    pturn = conv_dict["pturn"]
    for i, line in conv_dict["conversation"]:
        if i < 2: # skip first 2 lines of dialogue
            continue 
        if pturn == 1:
            prompt = prompts["eval_prompts"]["index_consistency"].replace("%SCENARIO_DESC%", prompts["scenario"]) \
                                                                 .replace("%SPEAKER_ROLE%", prompts["agent1_role"]) \
                                                                 .replace("%CONVERSATION%", format_conversation(conv_dict["conversation"][:i])) \
                                                                 .replace("%SPEAKER_LINE%", line)
            if config['verbose']:
                print(prompt)
            output = completion_create(config['eval_model'], config, prompt)
            index_list = extract_list(output)
            conv_dict['eval_index_consistency'].append((i, output))
            conv_dict['P1_index_consistency_score'] += len(index_list)
            p1_utterances += i // 2
            pturn = 2
        elif pturn == 2:
            if both_agents:
                prompt = prompts["eval_prompts"]["index_consistency"].replace("%SCENARIO_DESC%", prompts["scenario"]) \
                                                                     .replace("%SPEAKER_ROLE%", prompts["agent2_role"]) \
                                                                     .replace("%CONVERSATION%", format_conversation(conv_dict["conversation"][:i])) \
                                                                     .replace("%SPEAKER_LINE%", line)
                if config['verbose']:
                    print(prompt)
                output = completion_create(config['eval_model'], config, prompt)
                index_list = extract_list(output)
                conv_dict['eval_index_consistency'].append((i, output))
                conv_dict['P2_index_consistency_score'] += len(index_list)
                p2_utterances += i // 2
            pturn = 1

    if p1_utterances > 0:
        conv_dict['P1_index_consistency_score'] /= p1_utterances
        conv_dict['P1_index_consistency_score'] = 1 - conv_dict['P1_index_consistency_score']
    if p2_utterances > 0 and both_agents:
        conv_dict['P2_index_consistency_score'] /= p2_utterances
        conv_dict['P2_index_consistency_score'] = 1 - conv_dict['P2_index_consistency_score']

    return conv_dict

In [16]:
def eval_index_background_consistency(conv_dict, both_agents=False):
    conv_dict['eval_index_consistency'] = []
    conv_dict['P1_index_consistency_score'] = 0
    if both_agents:
        conv_dict['P2_index_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0
    pturn = conv_dict["pturn"]
    for i, line in conv_dict["conversation"]:
        if i < 2: # skip first 2 lines of dialogue
            continue 
        if pturn == 1:
            prompt = prompts["eval_prompts"]["index_consistency_background"].replace("%SCENARIO_DESC%", prompts["scenario"]) \
                                                                 .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                 .replace("%SPEAKER_ROLE%", prompts["agent1_role"]) \
                                                                 .replace("%CONVERSATION%", format_conversation(conv_dict["conversation"][:i])) \
                                                                 .replace("%SPEAKER_LINE%", line)
            if config['verbose']:
                print(prompt)
            output = completion_create(config['eval_model'], config, prompt)
            index_list = extract_list(output)
            conv_dict['eval_index_consistency'].append((i, output))
            conv_dict['P1_index_consistency_score'] += len(index_list)
            p1_utterances += i // 2
            pturn = 2
        elif pturn == 2:
            if both_agents:
                prompt = prompts["eval_prompts"]["index_consistency_background"].replace("%SCENARIO_DESC%", prompts["scenario"]) \
                                                                     .replace("%SPEAKER_BACKSTORY%", conv_dict["P2"]) \
                                                                     .replace("%SPEAKER_ROLE%", prompts["agent2_role"]) \
                                                                     .replace("%CONVERSATION%", format_conversation(conv_dict["conversation"][:i])) \
                                                                     .replace("%SPEAKER_LINE%", line)
                if config['verbose']:
                    print(prompt)
                output = completion_create(config['eval_model'], config, prompt)
                index_list = extract_list(output)
                conv_dict['eval_index_consistency'].append((i, output))
                conv_dict['P2_index_consistency_score'] += len(index_list)
                p2_utterances += i // 2
            pturn = 1

    if p1_utterances > 0:
        conv_dict['P1_index_consistency_score'] /= p1_utterances
        conv_dict['P1_index_consistency_score'] = 1 - conv_dict['P1_index_consistency_score']
    if p2_utterances > 0 and both_agents:
        conv_dict['P2_index_consistency_score'] /= p2_utterances
        conv_dict['P2_index_consistency_score'] = 1 - conv_dict['P2_index_consistency_score']

    return conv_dict

In [17]:
def eval_prompt_consistency(conv_dict):
    conv_dict['eval_prompt_consistency'] = {}
    conv_dict['P1_prompt_consistency_scores'] = {}
    p1_utterances = {}
    
    for key in ["strategy_consistency", "background_consistency", "combined_prompt_consistency"]:
        conv_dict['eval_prompt_consistency'][key] = []
        conv_dict['P1_prompt_consistency_scores'][key] = 0
        p1_utterances[key] = 0

    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            for key in ["strategy_consistency", "background_consistency", "combined_prompt_consistency"]:
                prompt = eval_prompts[key].replace("%SCENARIO_DESC", 'There is a Patient in conversation with a Therapist.') \
                                          .replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                          .replace("%SPEAKER_LINE%", convo_line)
                if config.get('verbose', False):
                    print(prompt)
                output = completion_create(config['eval_model'], config, prompt)
                conv_dict['eval_prompt_consistency'][key].append((line_number, output))
                if "YES" not in output:  # no contradiction
                    conv_dict['P1_prompt_consistency_scores'][key] += 1
                p1_utterances[key] += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1

    for key in ["strategy_consistency", "background_consistency", "combined_prompt_consistency"]:
        if p1_utterances[key] > 0:
            conv_dict['P1_prompt_consistency_scores'][key] /= p1_utterances[key]

    print(conv_dict)
    return conv_dict

In [18]:
config['eval_model']

'Llama-3.1-70B-Instruct'

In [ ]:
%%capture 

test_convs = []
for conversation in data:
    test_convs.append(eval_index_background_consistency(conversation))
    with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/exp/04.22.25/indexbackground_llama70b_Llama-3.1-8B-Instruct_0_500.json", 'w') as f:
        json.dump(test_convs, f, indent=4)  

2025-04-27 07:12:37,454	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


INFO 04-27 07:12:50 config.py:542] This model supports multiple tasks: {'score', 'generate', 'reward', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 04-27 07:12:51 config.py:1401] Defaulting to use mp for distributed inference
INFO 04-27 07:12:51 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Meta-Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=12880, download_dir='/raid/users/ryan_cheng/models/', load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoi

KeyboardInterrupt: 

INFO 04-27 07:23:16 multiproc_worker_utils.py:128] Killing local vLLM worker processes


(raylet) [2025-04-27 07:23:17,610 E 1559257 1559287] file_system_monitor.cc:116: /raid/users/ryan_cheng/tmp/ray/session_2025-04-27_07-12-35_532220_1540803 is over 95% full, available space: 356.497 GB; capacity: 28500 GB. Object creation will fail if spilling is required.
(raylet) [2025-04-27 07:23:27,614 E 1559257 1559287] file_system_monitor.cc:116: /raid/users/ryan_cheng/tmp/ray/session_2025-04-27_07-12-35_532220_1540803 is over 95% full, available space: 356.497 GB; capacity: 28500 GB. Object creation will fail if spilling is required.
(raylet) [2025-04-27 07:23:37,619 E 1559257 1559287] file_system_monitor.cc:116: /raid/users/ryan_cheng/tmp/ray/session_2025-04-27_07-12-35_532220_1540803 is over 95% full, available space: 356.497 GB; capacity: 28500 GB. Object creation will fail if spilling is required.
(raylet) [2025-04-27 07:23:47,623 E 1559257 1559287] file_system_monitor.cc:116: /raid/users/ryan_cheng/tmp/ray/session_2025-04-27_07-12-35_532220_1540803 is over 95% full, availabl

57 min 21 sec Llama-3.1-70B-Instruct

In [ ]:
test_convs = []
for conversation in data:
    test_convs.append(eval_index_consistency(conversation))
    with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/exp/04.22.25/index_llama70b_Llama-3.1-8B-Instruct_0_500.json", 'w') as f:
        json.dump(test_convs, f, indent=4)  

(raylet) [2025-04-27 06:58:11,702 E 1168096 1168128] file_system_monitor.cc:116: /raid/users/ryan_cheng/tmp/ray/session_2025-04-27_06-49-29_278705_3512463 is over 95% full, available space: 356.537 GB; capacity: 28500 GB. Object creation will fail if spilling is required.
(raylet) [2025-04-27 06:58:21,707 E 1168096 1168128] file_system_monitor.cc:116: /raid/users/ryan_cheng/tmp/ray/session_2025-04-27_06-49-29_278705_3512463 is over 95% full, available space: 356.537 GB; capacity: 28500 GB. Object creation will fail if spilling is required.
(raylet) [2025-04-27 06:58:31,710 E 1168096 1168128] file_system_monitor.cc:116: /raid/users/ryan_cheng/tmp/ray/session_2025-04-27_06-49-29_278705_3512463 is over 95% full, available space: 356.537 GB; capacity: 28500 GB. Object creation will fail if spilling is required.
(raylet) [2025-04-27 06:58:41,717 E 1168096 1168128] file_system_monitor.cc:116: /raid/users/ryan_cheng/tmp/ray/session_2025-04-27_06-49-29_278705_3512463 is over 95% full, availabl

In [ ]:
test_convs = []
for conversation in data:
    test_convs.append(eval_prompt_consistency(conversation))
    with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/exp/04.22.25/llama8beval_Llama-3.1-8B-Instruct_0_500.json", 'w') as f:
        json.dump(test_convs, f, indent=4)  



In [ ]:
example_conv

In [ ]:
config_therapy

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    p1_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = config_therapy["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    print(conv_dict)

    return conv_dict

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    p1_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = config_therapy["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    print(conv_dict)

    return conv_dict

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    conv_dict['P2_prompt_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        if pturn == 1:
            prompt = prompts["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", prompts["agent1_role"]) \
                                                                  .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                  .replace("%SPEAKER_LINE%", line)
            if config['verbose']:
                print(prompt)
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        else:
            prompt = prompts["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", prompts["agent2_role"]) \
                                                                  .replace("%SPEAKER_BACKSTORY%", conv_dict["P2"]) \
                                                                  .replace("%SPEAKER_LINE%", line)
            if config['verbose']:
                print(prompt)
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P2_prompt_consistency_score'] += 1
            p2_utterances += 1
            pturn = 1
    
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    if p2_utterances > 0:
        conv_dict['P2_prompt_consistency_score'] /= p2_utterances


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google/gemma-2b-it')

In [ ]:
messages = [
    {"role": "user", "content": 'hello world'}
]

In [ ]:
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)